In [2]:
from pyspark import SparkContext
sc = SparkContext()
sc.addPyFile("graphframes-0.7.0-spark2.4-s_2.11.jar")

from graphframes import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()
spark

In [3]:
sum = 0
rdd = sc.parallelize(range(20),2) 
def update_sum(x):
    global sum
    sum += x
rdd.foreach(update_sum)
print(sum)

0


In [4]:
sum = sc.accumulator(0)
rdd = sc.parallelize(range(20),2) 
def update_sum(x):
    global sum
    sum += x
rdd.foreach(update_sum)
print(sum)

190


In [8]:
A = sc.parallelize(range(1, 1000))
t = 100
B = A.filter(lambda x: x * x > t )
B.cache()
t = 200
C = B.filter(lambda x: x * x < t )
print (C.count())

4


In [19]:
rdd1 = sc.parallelize([5,0,0,3])
rdd2 = sc.parallelize([3,4,6,2])


def ele_wise_add(rdd1, rdd2): 
    return rdd1.zip(rdd2).map(lambda x: x[0] + x[1])

rdd3 = ele_wise_add(rdd1, rdd2)
print(rdd3.collect())
# Expected output: [8,4,6,5]

[8, 4, 6, 5]


In [3]:
from graphframes import *

# Vertices DataFrame
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("c", "a", "friend"),
 ("b", "c", "follow"),
 ("d", "a", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("f", "b", "follow"),
 ("c", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "d", "friend"),
 ("g", "c", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

# FILL IN YOUR CODE HERE

In [14]:
e2 = e.filter("relationship = 'follow'")
g2 = GraphFrame(v, e2)
zombies = g2.find("(a)-[]->(b); !(b)-[]->(a)") \
            .groupBy("a") \
            .count() \
            .orderBy('count', ascending = False) \
            .select("a.name")

zombies.show(1)

+-----+
| name|
+-----+
|Fanny|
+-----+
only showing top 1 row



In [92]:
e = sc.parallelize([(1,2),(1,3),(1,4),(2,3),(3,1)]).toDF(["src","dst"])

e1 = e.withColumnRenamed('src', 'x').withColumnRenamed('dst','y')
e2 = e.withColumnRenamed('src', 'y1').withColumnRenamed('dst','z')

e3 = e.withColumnRenamed('src', 'y').withColumnRenamed('dst','y1')

result = e1.join(e3, 'y').join(e2, 'y1') \
                         .withColumnRenamed('z', 'x1') \
                         .withColumnRenamed('y1', 'z') \
                         .filter("x == x1 AND x < y AND y < z") 

result.select('x','y','z').show()

+---+---+---+
|  x|  y|  z|
+---+---+---+
|  1|  2|  3|
+---+---+---+



In [15]:
import heapq

# Example to illustrate: We have 15 numbers, 3 works(3 partitions) to find 4 largest numbers
data = [34, 67, 21, 56, 47, 89, 12, 44, 74, 43, 26, 12, 5, 8, 85]
# number of works = 3 
A = sc.parallelize(data, 3)


def find_k_max(iterator):
    return heapq.nlargest(4, iterator)

# Find the k largest  integers in each partition and collect the result
collection = A.mapPartitions(find_k_max).collect()
print(collection)
# Calculate the k largest integers in the collection
result = heapq.nlargest(4, collection)

print(result)

[67, 56, 47, 34, 89, 74, 44, 43, 85, 26, 12, 8]
[89, 85, 74, 67]
